In [1]:
from lxml import etree
from io import StringIO
from lxml import etree
from io import StringIO
import sys, re, os, pdb, pickle
import pandas as pd

root_data_dir = '/Users/lauramiron/Large_files/CTData/'
xml_dir = root_data_dir+'AllPublicXML/'
results_dir = 'results/'
val_results_csv = results_dir+'xml_val_results.csv'

xml_files = [os.path.join(root, f) for root, dirs, files in os.walk(xml_dir) for f in files if f.split('.')[1]=='xml']
NUM_RECORDS = len(xml_files)
print_progress=True

In [2]:
def load_xml(filename_xml):
    with open(filename_xml, 'r') as xml_file:
        xml_to_check = xml_file.read()
    try:
        doc = etree.parse(StringIO(xml_to_check))
        return doc
    except IOError:
        return None
    except etree.XMLSyntaxError as err:
        return None

In [9]:
newline_pattern = r'([^\n])[\t ]*\n[\t ]*([^\n])'
character_class = '[\u0000-\u2BFF]'
loose_pattern = '^Inclusion Criteria:\s*'+'('+character_class+'+)'+'Exclusion Criteria:\s*'+'('+character_class+'*)$'
bullets_pattern = r'(?: {10}- {2}[\u0000-\u2BFF]*\n{2})*(?: {10}- {2}[\u0000-\u2BFF]*)'
strict_pattern = '^Inclusion Criteria:\s*'+'('+bullets_pattern+'+)'+'Exclusion Criteria:\s*'+'('+bullets_pattern+'*)$'
pattern = r'^\s*Inclusion Criteria:\s*(( *-[^\n]*)\n\n)*( *-[^\n]*)\s*Exclusion Criteria:\s*(( *-[^\n]*)\n\n)*( *-[^\n]*)\s*$'


num_missing = 0
num_good = 0
num_no_bullets = 0
num_bad_headers = 0
for count, xmlf in enumerate(xml_files):
    if count%10000 == 0: print(f'{count}/{len(xml_files)}')
    xml_doc = load_xml(xmlf)
    try:
        elig_elem = xml_doc.find('eligibility').find('criteria').find('textblock')
    except:
        num_missing += 1
        continue
    if elig_elem == None:
        num_missing += 1
        continue
    etext = re.sub(newline_pattern,r'\1 \2',elig_elem.text).strip()
    matches = re.search(pattern,etext)
    if matches != None:
        num_good += 1
        continue
    matches = re.search(loose_pattern, etext)
    if matches != None:
        num_no_bullets += 1
        continue
    num_bad_headers += 1
    
    
print(f'Records with correctly formatted eligibility criteria: {num_good}, {float(num_good)/len(xml_files):.2%}')
print(f'Records missing eligibility criteria: {num_missing}, {float(num_missing)/len(xml_files):.2%}')
print(f'Records with correct EC headers but criteria are a bulleted list: {num_no_bullets}, {float(num_no_bullets)/len(xml_files):.2%}')
print(f'Records with EC with missing or malformed headers: {num_bad_headers}, {float(num_bad_headers)/len(xml_files):.2%}')

0/302091
10000/302091
20000/302091
30000/302091
40000/302091
50000/302091
60000/302091
70000/302091
80000/302091
90000/302091
100000/302091
110000/302091
120000/302091
130000/302091
140000/302091
150000/302091
160000/302091
170000/302091
180000/302091
190000/302091
200000/302091
210000/302091
220000/302091
230000/302091
240000/302091
250000/302091
260000/302091
270000/302091
280000/302091
290000/302091
300000/302091
Records with correctly formatted eligibility criteria: 183312, 60.68%
Records missing eligibility criteria: 874, 0.29%
Records with correct EC headers but criteria are a bulleted list: 73779, 0.24422773270305967
Records with EC with missing or malformed headers: 44126, 14.61%
